<a href="https://colab.research.google.com/github/harnalashok/deeplearning-sequences/blob/main/time_series_modeling_with_attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Last amended: 9th July, 2021
# Ref: https://github.com/philipperemy/keras-attention-mechanism

## Install library

In [ ]:
# 0.0 Install attention
! pip install attention

## Call libraries

In [2]:
# 1.0 Usual libraries
import numpy as np

# 1.1 tensorflow related libraries
from tensorflow.keras import Input
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.models import load_model, Model
from attention import Attention

In [12]:
# 1.2
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

### Generate some sample data
Data has just one feature time-value

In [3]:
# 2.0 Take past 10 time steps and make prediction for the next
#       time step
num_samples, time_steps, input_dim, output_dim = 100, 10, 1, 1
data_x = np.random.uniform(size=(num_samples, time_steps, input_dim))
data_y = np.random.uniform(size=(num_samples, output_dim))

In [16]:
# 2.1 Observe data
print(data_x.shape)
print(data_x[:2])   # Show 2 sets of 10 time-steps each


(100, 10, 1)
[[[0.74432016]
  [0.08340127]
  [0.86140793]
  [0.20537519]
  [0.04363293]
  [0.31972668]
  [0.5667418 ]
  [0.25163508]
  [0.21397688]
  [0.10505855]]

 [[0.140055  ]
  [0.03418227]
  [0.4986226 ]
  [0.99729447]
  [0.06554542]
  [0.42046355]
  [0.23819792]
  [0.21301984]
  [0.4686359 ]
  [0.57741368]]]


In [ ]:
# 2.2 Show next 4-values to be predicted
print(data_y[:4])   # Show 4 values

### Our model

In [6]:
# 3.0 Define the model.
model_input = Input(shape=(time_steps, input_dim))
x = LSTM(64, return_sequences=True)(model_input)
x = Attention(32)(x)
x = Dense(1)(x)
model = Model(model_input, x)

In [ ]:
# 3.1 Compile and print model summary
model.compile(loss='mae', optimizer='adam')
print(model.summary())

In [ ]:
# 3.2 Start learning
model.fit(data_x, data_y, epochs=10)

In [ ]:
# 3.3 Make predictions
pred1 = model.predict(data_x)

### Save model for future

In [ ]:
# 4.0 Save model for future
model.save('test_model.h5')

In [ ]:
# 4.1 Delete earlier model
del model

# 4.2 Load the saved model
model_h5 = load_model('test_model.h5')

In [ ]:
# 4.3 Make predictions with the sabed model
pred2 = model_h5.predict(data_x)

In [ ]:
# 4.4 Compare saved model predictions 
#      with predictions made before saving

np.testing.assert_almost_equal(pred1, pred2)
print('Success.')

In [ ]:
########## I am done ###############